In [1]:
import pandas as pd
import numpy as np
import glob

files = glob.glob(r'\\project-best-PL21-22-transfers\\data\\transfer'+'\\*.csv')

df = pd.DataFrame()
for f in files:
    csv = pd.read_csv(f)
    df = df.append(csv)

In [2]:
df['league_name'] = df['league_name'].replace({'1 Bundesliga': 'Bundesliga', 
                                               'Primera Division': 'LaLiga',
                                               'Liga Nos':'Liga Portugal'
                                              })
df = df[df['fee_cleaned'].notna()]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51040 entries, 0 to 14626
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   club_name           51040 non-null  object 
 1   player_name         51040 non-null  object 
 2   age                 51030 non-null  float64
 3   position            51040 non-null  object 
 4   club_involved_name  51040 non-null  object 
 5   fee                 51040 non-null  object 
 6   transfer_movement   51040 non-null  object 
 7   transfer_period     51040 non-null  object 
 8   fee_cleaned         51040 non-null  float64
 9   league_name         51040 non-null  object 
 10  year                51040 non-null  int64  
 11  season              51040 non-null  object 
dtypes: float64(2), int64(1), object(9)
memory usage: 5.1+ MB


In [3]:
hist_ex = df.groupby(['league_name','year','transfer_movement'])['fee_cleaned'].sum().reset_index()
hist_ex = hist_ex.pivot(index=['league_name','transfer_movement'], columns='year')['fee_cleaned']
hist_ex.query("transfer_movement == 'in'", inplace=True)
hist_ex

,year,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
league_name,transfer_movement,,,,,,,,,,,,,,,,,,,,,
Bundesliga,in,29.395,37.889,54.262,78.113,51.214,70.665,74.945,151.185,115.970,185.205,...,291.110,307.900,369.025,475.900,730.828,737.040,562.810,956.240,388.250,487.145
Eredivisie,in,6.394,10.674,14.697,15.980,16.215,39.010,38.286,48.922,48.865,58.560,...,49.648,40.990,42.645,57.925,59.120,99.450,97.965,121.224,98.566,64.375
LaLiga,in,28.227,45.078,66.602,78.405,257.062,342.725,262.914,293.361,461.561,316.908,...,167.120,452.205,541.550,595.020,530.495,916.330,1052.250,1525.400,425.700,380.222
Liga Portugal,in,2.500,1.000,1.750,3.000,4.500,15.074,10.710,14.700,39.520,46.535,...,56.710,102.986,122.207,97.630,128.760,96.077,118.455,175.253,201.390,102.070
Ligue 1,in,23.398,10.780,9.400,4.850,17.330,52.050,61.460,118.240,233.470,171.370,...,260.400,413.580,176.520,365.205,372.220,751.405,723.600,840.615,467.090,464.840
Premier League,in,61.653,86.086,136.933,205.148,178.093,252.264,372.315,315.957,485.090,580.460,...,775.025,924.407,1233.705,1472.650,1662.265,2178.220,1652.060,1797.295,1536.260,1694.610
Serie A,in,130.365,84.345,48.264,98.285,112.460,205.505,342.520,502.670,523.698,815.545,...,572.065,604.187,464.911,762.898,864.322,1064.266,1364.593,1454.612,980.227,809.072


In [4]:
hist_ex.to_csv("historical_transfer_expenditure.csv", index=True)

In [5]:
df.query("season == '2021/2022' and transfer_movement == 'in' and fee_cleaned == fee_cleaned", inplace=True)
recent_transfer = df[(df != 0).all(1)]
recent_transfer = recent_transfer.reset_index()

recent_transfer['player_id'] = (recent_transfer.groupby('league_name')['fee_cleaned']
                      .rank(ascending=True)
                      .astype(int))
recent_transfer

,index,club_name,player_name,age,position,club_involved_name,fee,transfer_movement,transfer_period,fee_cleaned,league_name,year,season,player_id
0,12888,Bayern Munich,Dayot Upamecano,22.0,Centre-Back,RB Leipzig,€42.50m,in,Summer,42.50,Bundesliga,2021,2021/2022,90
1,12889,Bayern Munich,Marcel Sabitzer,27.0,Central Midfield,RB Leipzig,€15.00m,in,Summer,15.00,Bundesliga,2021,2021/2022,80
2,12910,RB Leipzig,André Silva,25.0,Centre-Forward,E. Frankfurt,€23.00m,in,Summer,23.00,Bundesliga,2021,2021/2022,87
3,12911,RB Leipzig,Josko Gvardiol,19.0,Centre-Back,Dinamo Zagreb,€18.80m,in,Summer,18.80,Bundesliga,2021,2021/2022,86
4,12912,RB Leipzig,Angeliño,24.0,Left-Back,Man City,€18.00m,in,Summer,18.00,Bundesliga,2021,2021/2022,85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,14545,SC Heerenveen,Thom Haye,26.0,Defensive Midfield,NAC Breda,€750Th.,in,Winter,0.75,Eredivisie,2021,2021/2022,13
565,14579,PSV Eindhoven,Joey Veerman,23.0,Central Midfield,Heerenveen,€6.00m,in,Winter,6.00,Eredivisie,2021,2021/2022,36
566,14603,FC Utrecht,Henk Veerman,30.0,Centre-Forward,Heerenveen,€350Th.,in,Winter,0.35,Eredivisie,2021,2021/2022,5
567,14619,Willem II Tilburg,Thijs Oosting,21.0,Attacking Midfield,AZ Alkmaar,€1.00m,in,Winter,1.00,Eredivisie,2021,2021/2022,19


In [6]:
recent_transfer.to_csv("recent_uefa_transfer.csv", index=True)

In [7]:
recent_transfer.groupby('league_name')['fee_cleaned'].describe()

,count,mean,std,min,25%,50%,75%,max
league_name,,,,,,,,
Bundesliga,90.0,5.412722,7.328060,0.085,0.5250,2.50,7.075,42.5
Eredivisie,37.0,1.739865,2.205011,0.100,0.4500,1.00,2.000,12.0
LaLiga,66.0,5.760939,9.468671,0.200,0.8875,2.50,5.500,55.0
Liga Portugal,38.0,2.686053,3.911328,0.050,0.3125,1.24,2.425,17.0
Ligue 1,78.0,5.959487,8.518152,0.200,1.5000,3.50,7.000,66.5
Premier League,99.0,17.117273,19.911185,0.300,5.0000,11.00,23.170,117.5
Serie A,161.0,5.025292,8.651799,0.007,0.6000,2.50,5.500,81.6


In [8]:
premier_league_transfer = recent_transfer
premier_league_transfer.query("league_name == 'Premier League' and transfer_period == 'Summer'", inplace=True)

In [9]:
premier_league_transfer.to_csv("recent_pl_transfer.csv", index=True)